# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Final Project: Part 2 - Feature Extraction


In any practical machine learning problem, the data preparation and feature extraction stages are the most important and time-consuming. The final project exposes you to a real-world dataset. In this part of the final project, you are responsible to creating features that will be meaningful for prediction. Features are evaluated based on Information Gain, which you implemented in [Assignment 2](../assignment-2/assignment-2.ipynb).

Here is what will work well in this project:

* Extract some sample data, load it in [R](https://www.r-project.org), and do some intial analysis. Feel free to build models there to get a feel for the best features.
* Join the different tables--they are there for a reason. 
* Get creative.
* Read some of the Kaggle competition forums and kernels. 

Here is what will NOT work:

* Do not use only the features as provided in application_train.
* Do not try implementing new learning algorithm in order to generate features. If you find something that works, investigate what features were helpful and add the features. 
* Do not build lookup tables "embeddings" or other things you might have read about but were not covered in class. 
* Do not try to build a kernel matrix on all pairs. Re-evaluate the kernel instead.

In [37]:
require './assignment_lib'

false

In [38]:
dir = "/home/dataset"
$dev_db = SQLite3::Database.new "#{dir}/credit_risk_data_dev.db", results_as_hash: true, readonly: true

#<SQLite3::Database:0x000000000bba1ca0 @tracefunc=nil, @authorizer=nil, @encoding=nil, @busy_handler=nil, @collations={}, @functions={}, @results_as_hash=true, @type_translation=nil, @readonly=true>

## Question 1.1 (10 Points)

Implement ```create_dataset``` which runs an SQL query on a database and constructs a dataset like those we have used in this course. Add an ```id``` field for the ```SK_ID_CURR``` and store the ```TARGET``` in ```label```. 

If the query is:
```sql
select sk_id_curr, target, ext_source_1 from application_train  where ext_source_1 <> '' order by sk_id_curr;
```

then the result is:

```json
[
    {"label":1,"id":100002,"features":{"ext_source_1":0.08303696739132256}},
    {"label":0,"id":100015,"features":{"ext_source_1":0.7220444501416448}}
]
...
```
Note the features should not include the ID or Label. Feature keys should be lowercase and only contain keys fo which ```key.is_a? String``` returns true.



In [39]:
def create_dataset db, sql
  dataset = []
  db.execute sql do |row|
    # BEGIN YOUR CODE
#     puts row
    data_point = Hash.new
    data_point["features"] = Hash.new
    row.keys.each do |key|
      if key.is_a? String
        if key == "SK_ID_CURR"
          data_point["id"] = row[key]
        elsif key == "TARGET"
          data_point["label"] = row[key]
        else
          data_point["features"][key.downcase] = row[key]
        end
      end
    end
    dataset << data_point
    #END YOUR CODE
  end
#   puts dataset[0]
  return dataset
end

:create_dataset

In [40]:
def test_11()
  dataset = create_dataset $dev_db, "select sk_id_curr, target, ext_source_1 from application_train where ext_source_1 <> '' 
order by sk_id_curr limit 37"
  assert_equal 37, dataset.size
  assert_true(dataset[0]["features"].has_key? "ext_source_1")
  assert_equal(1, dataset[0]["features"].size)
  assert_equal(100002, dataset[0]["id"])  
  assert_in_delta(0.08303696, dataset[0]["features"]["ext_source_1"], 1e-4)
  assert_equal(1, dataset[0]["label"])    
end

test_11()

## Question 1.1 (20 points)

Copy and revise **your** information gain calculation for numeric and categorical features, from [Assignment 2](../assignment-2/assignment-2.ipynb). Copy the following implementations

* Class Distribution
* Entropy
* Information Gain after splitting
* Information gain for numerical features (fast version)


In [41]:
def class_distribution dataset
  # BEGIN YOUR CODE
  res = Hash.new {|h,k| h[k] = 0}
  group = dataset.group_by{|p| p["label"]}
  group.keys.each do |k|
    res[k] = group[k].length * 1.0 / dataset.length
  end
  return res
  #END YOUR CODE
end

:class_distribution

In [42]:
def entropy dist
  # BEGIN YOUR CODE
  entropy = 0
  sum = 0
  dist.keys.each do |k|
    sum += dist[k]
  end
  dist.keys.each do |k|
    pro = dist[k] * 1.0 / sum
    if pro != 0
      entropy -= pro * Math.log(pro)
    end
  end
  return entropy
  #END YOUR CODE
end

:entropy

In [43]:
def test_12_1()
  # Check that there are three classes
  dataset = create_dataset $dev_db, "select target, sk_id_curr, ext_source_1, flag_own_car from application_train where ext_source_1 <> ''"
  dist = class_distribution dataset
  h0 = entropy dist
  assert_in_delta(0.2686201883261589, h0, 1e-3)
end

test_12_1()

In [44]:
def information_gain h0, splits
  # BEGIN YOUR CODE
  ig = h0
  count = 0
  splits.keys.each do |k|
    count += splits[k].length
  end
  splits.keys.each do |k|
    subset_entropy = entropy(class_distribution(splits[k]))
    ig -= splits[k].length * subset_entropy / count
  end
  return ig
  #END YOUR CODE
end

:information_gain

In [45]:
def test_12_2()
  # Check that there are three classes
  dataset = create_dataset $dev_db, "select target, sk_id_curr, ext_source_1, flag_own_car from application_train where ext_source_1 <> ''"
  dist = class_distribution dataset
  h0 = entropy dist
  
  splits = dataset.group_by {|row| row["features"]["flag_own_car"]}
  ig = information_gain h0, splits
  assert_in_delta(0.0002206258541794237, ig, 1e-4)
end

test_12_2()

In [46]:
def find_split_point_numeric y, h0, fname
  ### BEGIN SOLUTION
  ig_max = 0
  t_max = nil
  
  x = []
  y.each do |r|
    if r["features"][fname] != ""
      x << r
    end
  end

  feature_groups = x.group_by {|r| r["features"].fetch(fname, 0.0)}
  counts_right = Hash.new {|h,k| h[k] = 0}
  counts_left = Hash.new {|h,k| h[k] = 0}
  v_left = 0.0
  v_right = x.size.to_f

  feature_groups.each_key do |t|
    counts = Hash.new {|h,k| h[k] = 0}  
    feature_groups[t].each do |r| 
      counts[r["label"]] += 1
      counts_right[r["label"]] += 1
    end
    feature_groups[t] = counts
  end
  
  thresholds = feature_groups.keys.sort
  t = thresholds.shift
  
  feature_groups[t].each_key do |k| 
    counts_left[k] += feature_groups[t][k]
    counts_right[k] -= feature_groups[t][k]
    v_left += feature_groups[t][k]
    v_right -= feature_groups[t][k]
  end
  
  thresholds.each.with_index do |t, i|
    p_left = v_left / x.size
    p_right = v_right / x.size
    
    d_left = Hash.new
    d_right = Hash.new
    counts_left.each_key {|k| d_left[k] = counts_left[k] / v_left}
    counts_right.each_key {|k| d_right[k] = counts_right[k] / v_right}
        
    h_left = entropy(d_left)
    h_right = entropy(d_right)    
    ig = h0 - (p_left * h_left + p_right * h_right)
    if ig > ig_max
      ig_max = ig
      t_max = t
    end

    feature_groups[t].each_key do |k| 
      counts_left[k] += feature_groups[t][k]
      counts_right[k] -= feature_groups[t][k]
      v_left += feature_groups[t][k]
      v_right -= feature_groups[t][k]
    end
  end

  return [t_max, ig_max]
  ### END SOLUTION
end

:find_split_point_numeric

In [47]:
def test_12_3()
  # Check that there are three classes
  dataset = create_dataset $dev_db, "select target, sk_id_curr, ext_source_1, flag_own_car from application_train where ext_source_1 <> ''"
  dist = class_distribution dataset
  h0 = entropy dist
  
  t, ig = find_split_point_numeric dataset, h0, "ext_source_1"
  assert_in_delta(0.009751743140812785, ig, 1e-4)
end

test_12_3()

## Question 2.1 (70 Points)

Using whatever external software you want (hosted on your own devices), provide 15+ different features that have information >= 0.005. You may to implement several cells below, so please insert them above the test. 

Features must only be derived from the database but you are free to write whatever SQL queries you want. You may create temporary tables, but the database is read-only.

Pay close attention to the following aspects of feature design:

* Normalization: Z-score, L2, Min-Max, etc.
* Sparsity / missing values
* Frequency: Information is easily fooled by features with many values.
* Joins: Some of the best features in this dataset combine two columns from different tables.
* Transformations: One-hot, Binning, Discretization, Non-linear transformation

In [48]:
#Add extra cells as needed

In [49]:
class Object
  def is_number?
    to_f.to_s == to_s || to_i.to_s == to_s
  end
end

:is_number?

In [50]:
def fill_mean dataset
  feature_means = Hash.new{|h,k| h[k] = 0.0}
  feature_counts = Hash.new{|h,k| h[k] = 0.0}
  feature_sums = Hash.new{|h,k| h[k] = 0.0}
  dataset.each do |p|
    p["features"].keys.each do |k|
      if p["features"][k] != "" && p["features"][k].is_number?
        feature_counts[k] += 1
        feature_sums[k] += p["features"][k]
      end
    end
  end
  
  feature_counts.keys.each do |k|
    feature_means[k] = feature_sums[k] / feature_counts[k]
  end
  
  dataset.each do |p|
    p["features"].keys.each do |k|
      if p["features"][k] == ""
        p["features"][k] = feature_means[k]
      end
    end
  end
  
end

:fill_mean

In [51]:
def mean x
  sum = 0
  cnt = 0
  x.each do |value|
    if value != ""
      sum += value
      cnt += 1
    end
  end
  sum / cnt
end

def stdev x
  m = mean x
  sum = 0
  cnt = 0
  x.each do |value|
    if value != ""
      sum += (value - m) ** 2.0
      cnt += 1
    end
  end
  
  Math.sqrt(sum / cnt)
end

:stdev

In [52]:
def create_zdatabase database
  zdatabase = database.clone
  zdatabase = database.collect do |r|
    u = r.clone
    u["features"] = r["features"].clone
    u
  end
  
  zdatabase_features = []
  zdatabase[0]["features"].each_key do |key|
    zdatabase_features << key
  end

  # BEGIN YOUR CODE
  means = Hash.new
  stdevs = Hash.new
  zdatabase_features.each do |f|
    if zdatabase.any?{|p| !p["features"][f].is_number?}
      next
    end
    rs = zdatabase.select {|r| r["features"].has_key? f}
    x = rs.collect {|r| r["features"][f]} 
    means[f] = mean x
    stdevs[f] = stdev x    
    rs.each {|r| r["features"][f] = r["features"][f] == "" ? 0 : ((r["features"][f] - means[f]) / stdevs[f])}
  end
  #END YOUR CODE
  return zdatabase
end

:create_zdatabase

In [53]:
def extract_features db
  dataset = create_dataset $dev_db, "SELECT
  application_train.sk_id_curr,
  application_train.target,
  application_train.ext_source_1,
  application_train.ext_source_2,
  application_train.ext_source_3,
  application_train.flag_own_car,
  application_train.flag_own_realty,
  application_train.cnt_children,
  application_train.name_contract_type,
  application_train.code_gender,
  application_train.name_type_suite,
  application_train.name_income_type,
  application_train.name_education_type,
  application_train.name_family_status,
  application_train.name_housing_type,
  application_train.occupation_type,
  application_train.AMT_ANNUITY,
  application_train.AMT_GOODS_PRICE,
  application_train.AMT_CREDIT,
  application_train.DAYS_BIRTH,
  application_train.DAYS_EMPLOYED,
  application_train.DAYS_ID_PUBLISH,
  AVG(previous_application.AMT_ANNUITY) AS prev_amt_annuity,
  AVG(previous_application.AMT_CREDIT) AS prev_amt_credit,
  AVG(previous_application.AMT_GOODS_PRICE) AS prev_amt_goods_price,
  AVG(bureau.AMT_ANNUITY) AS bu_amt_annuity,
  AVG(bureau.AMT_CREDIT_SUM_DEBT) AS amt_credit_sum_debt,
  AVG(bureau.DAYS_CREDIT) AS days_credit,
  AVG(bureau.AMT_CREDIT_SUM) AS amt_credit_sum,
  AVG(installments_payments.AMT_PAYMENT) AS amt_payment,
  AVG(installments_payments.DAYS_INSTALMENT) AS days_instalment
  FROM
  application_train
  LEFT JOIN bureau ON application_train.sk_id_curr = bureau.sk_id_curr
  LEFT JOIN previous_application ON application_train.sk_id_curr = previous_application.sk_id_curr
  LEFT JOIN installments_payments ON previous_application.sk_id_prev = installments_payments.sk_id_prev
  GROUP BY
  application_train.sk_id_curr"
  dataset = fill_mean dataset
  splitset = Array.new
  
  dataset.each do |row|
    h = Hash.new
    h["features"] = Hash.new
    h["id"] = row["id"].clone
    h["label"] = row["label"].clone
    
    h["features"]["ext_source_2"] = row["features"]["ext_source_2"]
    h["features"]["ext_source_3"] = row["features"]["ext_source_3"]
    h["features"]["ext_prd"] = 
    row["features"]["ext_source_1"] * row["features"]["ext_source_2"] * row["features"]["ext_source_3"]
    h["features"]["ext_avg"] = 
    (row["features"]["ext_source_1"] + row["features"]["ext_source_2"] + row["features"]["ext_source_3"]) / 3.0
    
    h["features"]["f_1"] = 
    row["features"]["flag_own_car"].to_s + row["features"]["flag_own_realty"].to_s + 
    row["features"]["name_income_type"].to_s + row["features"]["name_housing_type"].to_s
    
    h["features"]["f_2"] = 
    row["features"]["code_gender"].to_s + row["features"]["cnt_children"].to_s + 
    row["features"]["name_education_type"].to_s + row["features"]["name_family_status"].to_s

    h["features"]["f_3"] = 
    row["features"]["name_type_suite"].to_s + row["features"]["name_income_type"].to_s +
    row["features"]["name_housing_type"].to_s + row["features"]["name_contract_type"].to_s
    
    h["features"]["f_4"] = 
    row["features"]["occupation_type"].to_s + row["features"]["name_education_type"].to_s + 
    row["features"]["name_housing_type"].to_s
    
    h["features"]["days_compound"] = 
    (row["features"]["days_birth"].to_f)**2 * (row["features"]["days_employed"].to_f)**2 * 
    (row["features"]["days_credit"].to_f * row["features"]["days_id_publish"].to_f) / 10**15
    
    h["features"]["days_birth_ext"] = 
    (row["features"]["days_birth"].to_f)**2 * 
    row["features"]["ext_source_2"].to_f / 10**5
    
    h["features"]["days_credit_ext"] = 
    (row["features"]["days_credit"].to_f)**2 * 
    row["features"]["ext_source_2"].to_f / 10**5
    
    amt_annuity = row["features"]["amt_annuity"].to_f
    amt_credit = row["features"]["amt_credit"].to_f
    bu_amt_annuity = row["features"]["bu_amt_annuity"].to_f
    bu_amt_credit = row["features"]["amt_credit_sum"].to_f
    amt_credit_sum_debt = row["features"]["amt_credit_sum_debt"].to_f
    amt_credit_sum = row["features"]["amt_credit_sum"].to_f
    amt_goods_price = row["features"]["amt_goods_price"].to_f
    prev_amt_credit = row["features"]["prev_amt_credit"].to_f
    prev_amt_goods_price = row["features"]["prev_amt_goods_price"].to_f
    amt_payment = row["features"]["amt_payment"].to_f
    days_instalment = row["features"]["days_instalment"].to_f
    
    credit_to_annuity = 0.0
    if amt_annuity == 0.0
      credit_to_annuity = amt_credit
    else
      credit_to_annuity = amt_credit / amt_annuity
    end
    
    bu_credit_to_annuity = 0.0
    if bu_amt_annuity == 0.0
      bu_credit_to_annuity = bu_amt_credit
    else
      bu_credit_to_annuity = bu_amt_credit / bu_amt_annuity
    end
    
    if amt_credit_sum == 0.0
      h["features"]["debt_to_credit"] = (amt_credit_sum_debt**3) + h["features"]["ext_source_2"]
    else
      h["features"]["debt_to_credit"] = 
      ((amt_credit_sum_debt / amt_credit_sum)**3) + h["features"]["ext_source_2"]
    end
    
    credit_minus_price = amt_credit - amt_goods_price
    prev_credit_minus_price = prev_amt_credit - prev_amt_goods_price
    h["features"]["credit_minus_price_ext"] = 
    (((credit_minus_price + prev_credit_minus_price)) / row["features"]["ext_source_2"])
    
    payment_time = (amt_payment * days_instalment)
    h["features"]["payment_time_ext"] = 
    (-payment_time * h["features"]["ext_source_2"])
    
    h["features"]["credit_to_annuity_ext"] = 
    (credit_to_annuity * bu_credit_to_annuity * h["features"]["ext_source_3"]**3)

    splitset << h
  end
  splitset = create_zdatabase splitset 
  return splitset
end

:extract_features

In [54]:
extracted_dataset = extract_features($dev_db)
extracted_dataset[0]

{"features"=>{"ext_source_2"=>-1.309768177747291, "ext_source_3"=>-2.1509738534295213, "ext_prd"=>-1.5210519921268584, "ext_avg"=>-3.189769240615041, "f_1"=>"NYWorkingHouse / apartment", "f_2"=>"M0Secondary / secondary specialSingle / not married", "f_3"=>"UnaccompaniedWorkingHouse / apartmentCash loans", "f_4"=>"LaborersSecondary / secondary specialHouse / apartment", "days_compound"=>-0.3517662724499399, "days_birth_ext"=>-1.2428388551006582, "days_credit_ext"=>-0.559826440020761, "debt_to_credit"=>-0.009425163382915479, "credit_minus_price_ext"=>-0.0238578320005015, "payment_time_ext"=>-0.5498353477808251, "credit_to_annuity_ext"=>-0.21453089880644846}, "id"=>100002, "label"=>1}

In [55]:
assert_not_nil extracted_dataset
assert_equal 15334, extracted_dataset.size
assert_true(extracted_dataset.all? {|row| row["features"].size >= 8}, "At least 6 non-zero features per row")
assert_true(extracted_dataset.flat_map {|row| row["features"].keys}.uniq.size >= 15,  "At least 15 features")

In [56]:
assert_equal 15334, extracted_dataset.collect {|row| row["id"]}.uniq.size
assert_equal 2, extracted_dataset.collect {|row| row["label"]}.uniq.size

h0 = entropy(class_distribution(extracted_dataset))
assert_in_delta(0.2797684909805576, h0, 1e-3)

In [57]:
features = extracted_dataset.flat_map {|row| row["features"].keys}.uniq
numeric_features = features.select {|k| extracted_dataset.reject {|row| row["features"][k] == ""}.all? {|row| row["features"].fetch(k, 0.0).is_a? Numeric}}

assert_true(numeric_features.size >= 4, "At least 4 numeric features")
def test_ig_numeric extracted_dataset, h0, test_feature1
  t, ig = find_split_point_numeric extracted_dataset, h0, test_feature1
  assert_true(ig >= 0.005, "Expected information gain for '#{test_feature1}' > 0.005")
  return test_feature1
end

test_ig_numeric extracted_dataset, h0, numeric_features[0]

"ext_source_2"

In [58]:
test_ig_numeric extracted_dataset, h0, numeric_features[1]

"ext_source_3"

In [59]:
test_ig_numeric extracted_dataset, h0, numeric_features[2]

"ext_prd"

In [60]:
3.upto(numeric_features.size - 1) do |i|
  test_ig_numeric extracted_dataset, h0, numeric_features[i]
end

3

In [61]:
categorical_features = features.select {|k| extracted_dataset.all? {|row| row["features"].fetch(k, "").is_a? String}}

assert_true(categorical_features.size >= 4, "At least 4 categorical features")

def test_ig_categorical extracted_dataset, h0, test_feature1
  splits = extracted_dataset.group_by {|row| row["features"][test_feature1]}
  ig = information_gain h0, splits
  puts ig
  assert_true(ig >= 0.005, "Expected information gain for '#{test_feature1}' > 0.005")
  return test_feature1
end

test_ig_categorical extracted_dataset, h0, categorical_features[0]

0.005604922071159194


"f_1"

In [62]:
test_ig_categorical extracted_dataset, h0, categorical_features[1]

0.0069960045246263616


"f_2"

In [63]:
test_ig_categorical extracted_dataset, h0, categorical_features[2]

0.007391439933385516


"f_3"

In [64]:
3.upto(categorical_features.size - 1) do |i|
  test_ig_categorical extracted_dataset, h0, categorical_features[i]
end

0.010408435930001108


3